<a href="https://colab.research.google.com/github/devdaveddev/BOTDb/blob/main/DBbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import openai
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from textblob import TextBlob
import torch
import random
import numpy as np

# Initialize NLP models
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score


openai.api_key = "YOUR_OPENAI_API_KEY"


def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    response = requests.get(url).json()
    return response.get("extract", "No information found.")


def fetch_reddit_data(topic):
    headers = {"User-Agent": "AI-Debater"}
    url = f"https://www.reddit.com/r/debate/comments.json?q={topic}&sort=top"
    response = requests.get(url, headers=headers).json()
    posts = [post['data']['title'] for post in response['data']['children'][:5]]
    return posts


def fetch_reddit_cmv_data(topic):
    headers = {"User-Agent": "AI-Debater"}
    url = f"https://www.reddit.com/r/changemyview/comments.json?q={topic}&sort=top"
    response = requests.get(url, headers=headers).json()
    posts = [post['data']['title'] for post in response['data']['children'][:5]]
    return posts


def select_model():
    return "GPT-4"  # Alternatively, use Llama 2 or Mixtral


def validate_with_knowledge_graph(topic):
    return f"Validated factual information on {topic}."

#Chain-of-Thought reasoning
def generate_human_like_arguments(topic, debate_type):
    wikipedia_info = fetch_wikipedia_data(topic)
    reddit_info = fetch_reddit_data(topic)
    reddit_cmv_info = fetch_reddit_cmv_data(topic)
    fact_check = validate_with_knowledge_graph(topic)

    weights = np.array([
        [0.7, 0.1, 0.1, 0.1],  # Educated: More Wikipedia, less Reddit, AI
        [0.0, 0.0, 1.0, 0.0],  # Radical: Only AIs
        [0.2, 0.5, 0.2, 0.1]   # Casual: More Reddit, some Wikipedia, little AI
    ])
    weight_labels = {"educated": 0, "radical": 1, "casual": 2}
    w_wiki, w_reddit, w_reddit_cmv, w_ai = weights[weight_labels.get(debate_type, 0)]

    combined_context = (
        f"Step 1: Gathering reliable sources.\n"
        f"Wikipedia Info: {wikipedia_info * w_wiki}\n\n"
        f"Step 2: Analyzing user discussions.\n"
        f"Reddit Insights: {reddit_info * w_reddit}\n\n"
        f"Step 3: Evaluating counterarguments from Change My View.\n"
        f"Change My View Insights: {reddit_cmv_info * w_reddit_cmv}\n\n"
        f"Step 4: Fact-checking for validity.\n"
        f"Fact Check: {fact_check}"
    )

    response = openai.ChatCompletion.create(
        model=select_model(),
        temperature=1.2,
        messages=[
            {"role": "system", "content": "You generate arguments by merging factual information from Wikipedia, Reddit, Change My View discussions, knowledge graphs, and OpenAI-generated insights, ensuring a human-like debate experience with step-by-step logical reasoning."},
            {"role": "user", "content": f"Debate this topic with strong reasoning: {topic}\n\n{combined_context}"}
        ]
    )
    return response["choices"][0]["message"]["content"]


def analyze_bias(user_input):
    sentiment_score = bert_sentiment_analysis(user_input)
    if sentiment_score >= 4:
        return "Highly Positive (Potential Bias)"
    elif sentiment_score <= 1:
        return "Highly Negative (Potential Bias)"
    return "Neutral"


def ai_debate():
    debate_type = input("Choose debate style (educated, radical, casual): ")
    topic = input("Enter a topic to debate: ")

    print("\nFetching arguments...\n")

    while True:
        ai_argument = generate_human_like_arguments(topic, debate_type)
        print("\nAI's Argument:")
        print(ai_argument)

        print("\nEnter your counter-argument or type 'exit' to stop:")
        user_argument = input()

        if user_argument.lower() == 'exit':
            print("Debate session ended. Thanks for debating!")
            break

        bias_analysis = analyze_bias(user_argument)
        print("\nBias Analysis:", bias_analysis)

        topic = f"Counter to: {user_argument}"

if __name__ == "__main__":
    ai_debate()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from textblob import TextBlob
import torch
import random
import numpy as np
import os
import tensorflow as tf
from mistralai import Mistral

# Initialize NLP models
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score

mistral_api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=mistral_api_key)

# TensorFlow Reinforcement
class DebateRLModel:
    def __init__(self):
        self.model = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1, activation='linear')
        ])
        self.model.compile(optimizer='adam', loss='mse')
        self.memory = []

    def train(self, X, y):
        self.model.fit(np.array(X), np.array(y), epochs=10, verbose=0)

    def predict(self, X):
        return self.model.predict(np.array(X))

    def remember(self, state, reward):
        self.memory.append((state, reward))

    def replay(self):
        if len(self.memory) > 10:
            batch = random.sample(self.memory, 10)
            X, y = zip(*batch)
            self.train(X, y)

rl_model = DebateRLModel()

def fetch_data(url, headers=None):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Data fetch error: {e}")
        return {}

def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    return fetch_data(url).get("extract", "No information found.")

def fetch_stackexchange_data(topic):
    url = f"https://api.stackexchange.com/2.3/search?order=desc&sort=relevance&intitle={topic}&site=history"
    response = fetch_data(url)
    return [item["title"] for item in response.get("items", [])[:5]]

def fetch_reddit_data(topic):
    url = f"https://www.reddit.com/r/debaterelated/search.json?q={topic}&restrict_sr=1"
    headers = {'User-Agent': 'debate-bot'}
    response = fetch_data(url, headers)
    return [post["data"]["title"] for post in response.get("data", {}).get("children", [])[:5]]

def fetch_philosopher_data():
    philosophers = ["Socrates", "Plato", "Nietzsche", "Franz Kafka", "Neil deGrasse Tyson", "Alex O'Connor"]
    return {philosopher: fetch_wikipedia_data(philosopher) for philosopher in philosophers}

def validate_with_knowledge_graph(topic):
    return f"Validated factual information on {topic}."

def weighted_text(text, weight):
    return text[:int(len(text) * weight)] if isinstance(text, str) else text[:max(1, int(len(text) * weight))]

def update_weights(weights, feedback_score, learning_rate=0.1):
    new_weights = weights + learning_rate * (feedback_score - 3) * weights
    rl_model.remember(weights, new_weights)
    rl_model.replay()
    return new_weights
def add_philospher(philospher):


def assess_debate_response(response):
    criteria = ["Premise", "Propositions", "Arguments", "Rebuttals", "Evidence"]
    scores = {criterion: int(input(f"Rate the response on {criterion} (1-5): ")) for criterion in criteria}
    return scores

def generate_human_like_arguments(topic, debate_type, role, weights):
    wikipedia_info = fetch_wikipedia_data(topic)
    stackexchange_info = fetch_stackexchange_data(topic)
    reddit_info = fetch_reddit_data(topic)
    philosopher_data = fetch_philosopher_data()
    fact_check = validate_with_knowledge_graph(topic)

    weight_labels = {"educated": 0, "radical": 1, "casual": 2}
    w_wiki, w_stack, w_reddit, w_ai = weights[weight_labels.get(debate_type, 0)]

    combined_context = (
        f"Wikipedia Info: {weighted_text(wikipedia_info, w_wiki)}\n"
        f"StackExchange Insights: {weighted_text(' '.join(stackexchange_info), w_stack)}\n"
        f"Reddit Insights: {weighted_text(' '.join(reddit_info), w_reddit)}\n"
        f"Fact Check: {fact_check}\n"
        f"Philosopher Insights: {weighted_text(str(philosopher_data), w_ai)}"
    )

    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=[
            {"role": "system", "content": f"You are roleplaying as {role}. Follow structured reasoning."},
            {"role": "user", "content": f"Debate this topic: {topic}\n{combined_context}"}
        ]
    )
    return chat_response.choices[0].message.content, weights


In [ ]:
!pip install requests nltk transformers textblob torch tensorflow beautifulsoup4 datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet
from transformers import pipeline, BertForSequenceClassification, BertTokenizer, T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from textblob import TextBlob
import torch
import random
import numpy as np
import os
import tensorflow as tf
from bs4 import BeautifulSoup
from datasets import load_dataset

# Initialize NLP models
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
sia = SentimentIntensityAnalyzer()
summarizer = pipeline("summarization")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Fine-tuning hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

def fetch_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    return fetch_data(url).get("extract", "No information found.")

def fetch_stackexchange_data(topic):
    url = f"https://api.stackexchange.com/2.3/search?order=desc&sort=relevance&intitle={topic}&site=history"
    response = fetch_data(url)
    return [item["title"] for item in response.get("items", [])[:5]]

def fetch_reddit_data(topic):
    url = f"https://www.reddit.com/r/debaterelated/search.json?q={topic}&restrict_sr=1"
    headers = {'User-Agent': 'debate-bot'}
    response = fetch_data(url, headers=headers)
    return [post["data"]["title"] for post in response.get("data", {}).get("children", [])[:5]]

def fetch_philosopher_data():
    philosophers = ["Socrates", "Plato", "Nietzsche", "Franz Kafka", "Neil deGrasse Tyson", "Alex O'Connor"]
    return {philosopher: fetch_wikipedia_data(philosopher) for philosopher in philosophers}

def fetch_philpapers_data(query):
    api_key = "siFd5mcgjXhqB1Ir"
    user_id = "1772418"
    url = f"https://philpapers.org/philapi/search?searchstr={query}&apikey={api_key}&userid={user_id}"
    response = fetch_data(url)
    return [item["title"] for item in response.get("results", [])[:5]]

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score

# Transformer for Philosophical Style Adaptation
style_tokenizer = T5Tokenizer.from_pretrained("t5-small")
style_model = T5ForConditionalGeneration.from_pretrained("t5-small")

def adapt_style(text, philosopher):
    input_text = f"Rewrite as {philosopher}: {text}"
    inputs = style_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = style_model.generate(**inputs)
    return style_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Lexical Substitution for Philosophical Language Enhancement
def lexical_substitution(text):
    words = nltk.word_tokenize(text)
    new_text = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            new_text.append(synonyms[0].lemmas()[0].name())
        else:
            new_text.append(word)
    return ' '.join(new_text)

# ANN Model for Learning Debate Styles
class DebateANN(tf.keras.Model):
    def __init__(self):
        super(DebateANN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(5, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)

ann_model = DebateANN()
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TensorFlow Reinforcement Learning Model with LSTM
class DebateRLModel:
    def __init__(self):
        self.lstm_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(32, activation='relu', input_shape=(None, 5)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(5, activation='linear')
        ])
        self.lstm_model.compile(optimizer='adam', loss='mse')

        self.rl_model = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu', input_shape=(5,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(5, activation='linear')
        ])
        self.rl_model.compile(optimizer='adam', loss='mse')

        self.memory = []

    def train(self):
        if len(self.memory) > 100:
            batch = random.sample(self.memory, 100)
            X, y = zip(*batch)
            self.rl_model.fit(np.array(X), np.array(y), epochs=5, verbose=0)
            self.lstm_model.fit(np.array(X), np.array(y), epochs=5, verbose=0)

    def predict(self, X):
        rl_prediction = self.rl_model.predict(np.array([X]))[0]
        lstm_prediction = self.lstm_model.predict(np.array([X]))[0]
        ann_prediction = ann_model.predict(np.array([X]))[0]
        combined_prediction = (rl_prediction + lstm_prediction + ann_prediction) / 3
        return combined_prediction

    def remember(self, state, reward):
        self.memory.append((state, reward))
        if len(self.memory) > 1000:
            self.memory.pop(0)

rl_model = DebateRLModel()

#############################
# Interactive Debate Function
#############################

def philosophical_debate():
    philosophers = ["Socrates", "Plato", "Aristotle", "Nietzsche", "Descartes"]
    print("Choose a philosopher from the following list:")
    print(", ".join(philosophers))

    while True:
        selected_philosopher = input("Enter the name of your choice: ")
        if selected_philosopher in philosophers:
            break
        else:
            print("Invalid input. Please enter a name from the list.")

    print(f"You are debating with {selected_philosopher}. Enter your argument:")
    user_input = input()
    response = adapt_style(user_input, selected_philosopher)
    print(f"{selected_philosopher}: {response}")


def ai_debate():
    print("Enter your argument for an AI-generated response:")
    user_input = input()
    wikipedia_info = fetch_wikipedia_data(user_input)
    stackexchange_info = fetch_stackexchange_data(user_input)
    reddit_info = fetch_reddit_data(user_input)
    philpapers_info = fetch_philpapers_data(user_input)
    summary = summarizer(wikipedia_info, max_length=100, min_length=30, do_sample=False)
    print("AI Response:")
    print(summary[0]['summary_text'])
    print("StackExchange Discussions:", stackexchange_info)
    print("Reddit Discussions:", reddit_info)
    print("PhilPapers Discussions:", philpapers_info)

def interactive_debate():
    print("Welcome to the AI Debate System!")
    mode = input("Choose mode: 1 for Philosophical Debate, 2 for Independent AI Response: ")
    if mode == "1":
        philosophical_debate()
    elif mode == "2":
        ai_debate()
    else:
        print("Invalid choice.")

interactive_debate()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Welcome to the AI Debate System!
Choose mode: 1 for Philosophical Debate, 2 for Independent AI Response: 2
Enter your argument for an AI-generated response:
i am anti abortion give me points for pro abortion


Your max_length is set to 100, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


AI Response:
 No information found. No information was found on the phone line. No contact was made with the number of people in need of phone calls. No number was found .
StackExchange Discussions: []
Reddit Discussions: []
PhilPapers Discussions: []


In [ ]:
import requests
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer, BartForConditionalGeneration, BartTokenizer, TrainingArguments
from textblob import TextBlob
import torch
import random
import numpy as np
import os
import tensorflow as tf
from bs4 import BeautifulSoup
from datasets import load_dataset

# Initialize NLP models
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
sia = SentimentIntensityAnalyzer()

# Load pre-trained GPT-2 model and tokenizer for text generation
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
text_generator = pipeline("text-generation", model=gpt2_model, tokenizer=gpt2_tokenizer)

# Load pre-trained BART model and tokenizer for summarization
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Load BERT model for sentiment analysis
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Fine-tuning hyperparameters
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

def fetch_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return {}

def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    return fetch_data(url).get("extract", "No information found.")

def fetch_stackexchange_data(topic):
    url = f"https://api.stackexchange.com/2.3/search?order=desc&sort=relevance&intitle={topic}&site=history"
    response = fetch_data(url)
    return [item["title"] for item in response.get("items", [])[:5]]

def fetch_reddit_data(topic):
    url = f"https://www.reddit.com/r/debaterelated/search.json?q={topic}&restrict_sr=1"
    headers = {'User-Agent': 'debate-bot'}
    response = fetch_data(url, headers=headers)
    return [post["data"]["title"] for post in response.get("data", {}).get("children", [])[:5]]

def fetch_philosopher_data():
    philosophers = ["Socrates", "Plato", "Nietzsche", "Franz Kafka", "Neil deGrasse Tyson", "Alex O'Connor"]
    return {philosopher: fetch_wikipedia_data(philosopher) for philosopher in philosophers}

def fetch_philpapers_data(query):
    api_key = "siFd5mcgjXhqB1Ir"
    user_id = "1772418"
    url = f"https://philpapers.org/philapi/search?searchstr={query}&apikey={api_key}&userid={user_id}"
    response = fetch_data(url)
    return [item["title"] for item in response.get("results", [])[:5]]

def bert_sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = bert_model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = torch.argmax(scores).item()
    return sentiment_score

# Transformer for Philosophical Style Adaptation
style_tokenizer = T5Tokenizer.from_pretrained("t5-small")
style_model = T5ForConditionalGeneration.from_pretrained("t5-small")

def adapt_style(text, philosopher):
    input_text = f"Rewrite as {philosopher}: {text}"
    inputs = style_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = style_model.generate(**inputs)
    return style_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Lexical Substitution for Philosophical Language Enhancement
def lexical_substitution(text):
    words = nltk.word_tokenize(text)
    new_text = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            new_text.append(synonyms[0].lemmas()[0].name())
        else:
            new_text.append(word)
    return ' '.join(new_text)

# ANN Model for Learning Debate Styles
class DebateANN(tf.keras.Model):
    def __init__(self):
        super(DebateANN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(5, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)

ann_model = DebateANN()
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# TensorFlow Reinforcement Learning Model with LSTM
class DebateRLModel:
    def __init__(self):
        self.lstm_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(32, activation='relu', input_shape=(None, 5)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(5, activation='linear')
        ])
        self.lstm_model.compile(optimizer='adam', loss='mse')

        self.rl_model = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu', input_shape=(5,)),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(5, activation='linear')
        ])
        self.rl_model.compile(optimizer='adam', loss='mse')

        self.memory = []

    def train(self):
        if len(self.memory) > 100:
            batch = random.sample(self.memory, 100)
            X, y = zip(*batch)
            self.rl_model.fit(np.array(X), np.array(y), epochs=5, verbose=0)
            self.lstm_model.fit(np.array(X), np.array(y), epochs=5, verbose=0)

    def predict(self, X):
        rl_prediction = self.rl_model.predict(np.array([X]))[0]
        lstm_prediction = self.lstm_model.predict(np.array([X]))[0]
        ann_prediction = ann_model.predict(np.array([X]))[0]
        combined_prediction = (rl_prediction + lstm_prediction + ann_prediction) / 3
        return combined_prediction

    def remember(self, state, reward):
        self.memory.append((state, reward))
        if len(self.memory) > 1000:
            self.memory.pop(0)

rl_model = DebateRLModel()

def summarize(text, max_length=50):
    inputs = bart_tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=max_length, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def generate_pro_abortion_points(prompt):
    response = text_generator(prompt, max_length=200, num_return_sequences=1)
    generated_text = response[0]['generated_text']
    return summarize(generated_text)

#############################
# Interactive Debate Function
#############################

def philosophical_debate():
    philosophers = ["Socrates", "Plato", "Aristotle", "Nietzsche", "Descartes"]
    print("Choose a philosopher from the following list:")
    print(", ".join(philosophers))

    while True:
        selected_philosopher = input("Enter the name of your choice: ")
        if selected_philosopher in philosophers:
            break
        else:
            print("Invalid input. Please enter a name from the list.")

    print(f"You are debating with {selected_philosopher}. Enter your argument:")
    user_input = input()
    response = adapt_style(user_input, selected_philosopher)
    print(f"{selected_philosopher}: {response}")

def ai_debate():
    print("Enter your argument for an AI-generated response:")
    user_input = input()
    wikipedia_info = fetch_wikipedia_data(user_input)
    stackexchange_info = fetch_stackexchange_data(user_input)
    reddit_info = fetch_reddit_data(user_input)
    philpapers_info = fetch_philpapers_data(user_input)
    pro_abortion_points = generate_pro_abortion_points("I am anti-abortion. Give me points for pro-abortion.")
    print("AI Response:")
    print(pro_abortion_points)
    print("StackExchange Discussions:", stackexchange_info)
    print("Reddit Discussions:", reddit_info)
    print("PhilPapers Discussions:", philpapers_info)

def interactive_debate():
    print("Welcome to the AI Debate System!")
    mode = input("Choose mode: 1 for Philosophical Debate, 2 for Independent AI Response: ")
    if mode == "1":
        philosophical_debate()
    elif mode == "2":
        ai_debate()
    else:
        print("Invalid choice.")

interactive_debate()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Device set to use cpu


Welcome to the AI Debate System!
Choose mode: 1 for Philosophical Debate, 2 for Independent AI Response: 2
Enter your argument for an AI-generated response:
give me anti abortion points and debate with me


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI Response:
I know of at least five women who are using this same method to get laid. You wouldn't think they would've a problem with a pay cut. But they would, even in those circumstances, be willing to have their workers paid
StackExchange Discussions: []
Reddit Discussions: []
PhilPapers Discussions: []


In [ ]:
import requests
import nltk
import torch
import numpy as np
import random
import openai
import tensorflow as tf
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer

# Download necessary NLTK data
nltk.download("vader_lexicon")
nltk.download("punkt")
nltk.download("wordnet")

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# OpenAI GPT-4 Setup
openai.api_key = "YOUR_OPENAI_API_KEY"

def generate_gpt4_response(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        return f"GPT-4 Error: {str(e)}"

# Wikipedia API

def fetch_wikipedia_data(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    try:
        response = requests.get(url)
        return response.json().get("extract", "No information found.")
    except Exception as e:
        return f"Wikipedia Error: {str(e)}"

# StackExchange API

def fetch_stackexchange_data(topic):
    url = f"https://api.stackexchange.com/2.3/search?order=desc&sort=relevance&intitle={topic}&site=history"
    try:
        response = requests.get(url)
        data = response.json()
        return [item["title"] for item in data.get("items", [])[:5]] or ["No relevant discussions found."]
    except Exception as e:
        return [f"StackExchange Error: {str(e)}"]

# Reddit API (Public Search)

def fetch_reddit_data(topic):
    url = f"https://www.reddit.com/r/debaterelated/search.json?q={topic}&restrict_sr=1"
    headers = {'User-Agent': 'debate-bot'}
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        return [post["data"]["title"] for post in data.get("data", {}).get("children", [])[:5]] or ["No relevant posts found."]
    except Exception as e:
        return [f"Reddit Error: {str(e)}"]

# PhilPapers API

def fetch_philpapers_data(query):
    api_key = "siFd5mcgjXhqB1Ir"
    user_id = "1772418"
    url = f"https://philpapers.org/philapi/search?searchstr={query}&apikey={api_key}&userid={user_id}"
    try:
        response = requests.get(url)
        data = response.json()
        return [item["title"] for item in data.get("results", [])[:5]] or ["No relevant results found."]
    except Exception as e:
        return [f"PhilPapers API error: {str(e)}"]

# Dynamic Weighting System Using LSTM
class SourceWeightingModel(tf.keras.Model):
    def __init__(self):
        super(SourceWeightingModel, self).__init__()
        self.lstm = tf.keras.layers.LSTM(32, return_sequences=False)
        self.dense = tf.keras.layers.Dense(5, activation='softmax')

    def call(self, inputs):
        x = self.lstm(inputs)
        return self.dense(x)

weight_model = SourceWeightingModel()
weight_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Initial source weights
source_weights = np.array([0.25, 0.25, 0.2, 0.15, 0.15])

# Function to update weights dynamically based on user feedback
def update_weights(feedback):
    global source_weights
    gradient = np.array(feedback) - source_weights
    source_weights += 0.01 * gradient  # Learning rate of 0.1
    source_weights = np.clip(source_weights, 0, 1)
    source_weights /= np.sum(source_weights)  # Normalize

# Aggregating responses
def aggregate_response(topic):
    sources = [
        fetch_wikipedia_data(topic),
        fetch_stackexchange_data(topic),
        fetch_reddit_data(topic),
        fetch_philpapers_data(topic),
        generate_gpt4_response(topic)
    ]

    scores = np.random.rand(5)  # Simulated feedback for now
    weighted_scores = source_weights * scores
    best_source = np.argmax(weighted_scores)

    return sources[best_source]

# Interactive Debate
def interactive_debate():
    print("Welcome to the AI Debate System!")
    while True:
        user_input = input("Enter your argument (or type 'exit' to quit): ")
        if user_input.lower() == "exit":
            print("Exiting debate.")
            break
        response = aggregate_response(user_input)
        print(f"AI Response: {response}")
        feedback = list(map(float, input("Rate response accuracy (5 values, space-separated): ").split()))
        update_weights(feedback)

interactive_debate()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Welcome to the AI Debate System!
AI Response: No information found.
AI Response: No information found.
